### Часть 1

Составить таблицу со значениями валют относительно USD в определенные дни

In [1]:
%%capture
%pip install -r requirements.txt

In [2]:
import requests
import pandas as pd
import os
import typing
from datetime import date, timedelta
from dotenv import load_dotenv
from itertools import islice

load_dotenv()

KEY = os.getenv("API_KEY") # ключ в лежит .env, чтобы не раскрывать его в публичном репо

C:\Users\stray\AppData\Local\Temp\ipykernel_10876\3142475634.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
def get_historical_rate(historical_date: date) -> dict:
    """Получить исторические данные значений валют относительно USD"""
    
    if historical_date >= date.today():
        raise ValueError("Дата должна быть раньше сегодня")
    
    # если бы какие-то еще эндпоинты использовали, то имело смысл бы отдельный клиент сделать как класс, чтобы не повторять код
    # но тут незачем выдумывать, поэтому запрос прямо тут собран
    headers= {"apikey": KEY}
    url = "https://api.apilayer.com/currency_data/historical"
    params = {
        "date": str(historical_date)
    }
    r = requests.get(url=url, params=params, headers=headers)
    return r.json()

In [4]:
# рецепт из https://docs.python.org/3/library/itertools.html
def take(n: int, iterable: typing.Iterable) -> list:
    "Return first n items of the iterable as a list."
    return list(islice(iterable, n))

In [5]:
def get_date_range(start: date, end: date) -> list[date]:
    """Получить список дней в отрезке [start, end]"""
    return [start + timedelta(days=step) for step in range((end - start).days + 1)]

In [6]:
dates = get_date_range(date(2022, 1, 1), date(2022, 1, 7))
rates = [get_historical_rate(d) for d in dates]

In [7]:
# выделение значений первых 10 валют относительно USD
first_ten_rates = [dict(zip(take(10, rate["quotes"]), take(10, rate["quotes"].values()))) for rate in rates] 

In [8]:
data = {str(d): r for d, r in zip(dates, first_ten_rates)}

In [32]:
df = pd.DataFrame.from_dict(data, orient="index")
df.index.names = ["Date"]
df

,USDAED,USDAFN,USDALL,USDAMD,USDANG,USDAOA,USDARS,USDAUD,USDAWG,USDAZN
Date,,,,,,,,,,
2022-01-01,3.673042,103.750404,106.650403,480.234325,1.802591,550.590403,102.689974,1.375800,1.8000,1.703970
2022-01-02,3.672980,103.749983,106.650326,480.234325,1.802591,550.590435,102.664691,1.376179,1.8000,1.698647
2022-01-03,3.673103,103.750299,106.249880,481.590005,1.802973,554.203977,102.958502,1.389502,1.8005,1.701522
2022-01-04,3.673096,104.899369,105.874950,482.301630,1.801500,553.427992,103.104100,1.381105,1.8000,1.698131
2022-01-05,3.673102,105.250158,106.902530,483.164239,1.802797,552.652983,103.102801,1.384754,1.8005,1.691373
2022-01-06,3.673102,104.999962,107.350118,483.038449,1.802366,551.700853,103.228600,1.395982,1.8000,1.700773
2022-01-07,3.673042,105.000368,107.000368,483.788207,1.802362,550.928041,103.292284,1.392651,1.8005,1.703970


In [33]:
df.to_csv("result.csv")